# Number 3

## Derivation

Assume given a filtered probability space $(\Omega,\mathscr{F},(\mathscr{F}_t)_{t\in[0,T]}, \mathrm{P})$, where the filtration is generated by a Brownian motion $W_t$.

The market consists of two assets: a _riskless asset_ with price $B_t$ (e.g. money market account) and a _risky asset_ with price $S_t$ (e.g. FX Spot) given by a geometric Brownian motion:

\begin{align*}
&d B_t = r B_t dt,& &B_0=1,\qquad(B_t=e^{rt}),\\
&d S_t = S_t(\mu dt + \sigma d W_t),&  &S_0>0, 
  \qquad \left(S_t = S_0e^{\sigma W_t + \left(\mu-\frac{\sigma^2}{2}\right)t}\right).
\end{align*}


Also denote some parametrs:
1. $r_f$ - foreign risk free rate,
2. $K$ - the FX strike rate,
3. $T$ - maturity period.

And the model Garman-Kohlhagen has similar state like Black-Scholes model. The call and put price can be calculated by formula below:
$$
V^{\text{call}} = Se^{-r_f T}\Phi(d_1) - e^{-rT}K\Phi(d_2), \quad
V^{\text{put}} = e^{-rT} K\Phi(-d_2) -Se^{-r_f T}\Phi(-d_1),
$$

where $\Phi(x)$ is the standard normal distribution function, and
$$
d_1 = \frac{1}{\sigma\sqrt T} \left(\ln\frac{S}{K} + \left(r-r_f+\frac{\sigma^2}{2}\right) T\right) , 
d_2 = \frac{1}{\sigma\sqrt T} \left(\ln\frac{S}{K} + \left(r-r_f-\frac{\sigma^2}{2}\right) T\right).
$$

## The greeks

Delta 
$$\Delta^{call} = \displaystyle \frac{\partial V^{call}}{\partial S} = e^{-r_f T}\Phi(d_1)$$

Gamma
$$
\Gamma = \displaystyle\frac{\partial^{2} V}{\partial S^{2}} = \frac{e^{-r_f T}\Phi'(d_1)}{S\sigma\sqrt{T - t}}
$$

Theta
$$
\theta^{call} = \displaystyle \frac{\partial V^{call}}{\partial t} = 
r_fSe^{-r_f(T - t)}\Phi(d_1) -Se^{-r_f(T - t)}\frac{ \Phi'(d_1) \sigma}{2 \sqrt{T - t}} - rKe^{-r(T - t)}\Phi(d_2).
$$


In [410]:
from typing import Union
from dataclasses import dataclass
import numpy as np
import numpy.typing as npt
from scipy import stats

In [411]:
FloatArray = npt.NDArray[np.float_]
Floats = Union[float, FloatArray]

Here we make support class for future application. Names of classes is meaning what it names)

In [412]:
@dataclass
class MarketState:
    spot_price: Floats
    domestic_rate: Floats
    time: Floats = 0

@dataclass
class StockOption:
    strike_price: Floats
    expiration_time: Floats  # in years
    is_call: Union[bool, npt.NDArray[np.bool_]]
    
    def payoff(self, spot_price: Floats) -> Floats:
        call_payoff = np.maximum(0, spot_price - self.strike_price)
        put_payoff = np.maximum(0, self.strike_price - spot_price)
        return np.where(self.is_call, call_payoff, put_payoff)

@dataclass
class CallStockOption(StockOption):
    def __init__(self, strike_price, expiration_time):
        super().__init__(strike_price, expiration_time, True)
        
@dataclass
class PutStockOption(StockOption):
    def __init__(self, strike_price, expiration_time):
        super().__init__(strike_price, expiration_time, False)    

@dataclass
class GKParams:
    volatility: Floats
    foreign_rate: Floats


This block is realisation of $d_1$ and $d_2$ by formulas under. And `dt` function is helpful function for correct pricing becouse we can divide one zero at the formula.

In [413]:
def dt(option: StockOption, ms: MarketState):
    return np.maximum(option.expiration_time - ms.time, np.finfo(np.float64).eps)


def d1(option: StockOption, ms: MarketState, params: GKParams):
    return 1 / (params.volatility * np.sqrt(dt(option, ms)))\
                * (np.log(ms.spot_price / option.strike_price)
                   + (ms.domestic_rate - params.foreign_rate + params.volatility ** 2 / 2) * dt(option, ms))


def d2(option: StockOption, ms: MarketState, params: GKParams):
    return 1 / (params.volatility * np.sqrt(dt(option, ms)))\
                * (np.log(ms.spot_price / option.strike_price)
                   + (ms.domestic_rate - params.foreign_rate - params.volatility ** 2 / 2) * dt(option, ms))
    # return d1(option, ms, params) - params.volatility * np.sqrt(dt(option, ms))

This block is realisation of price of put/call options by formulas under.

In [414]:
def price(option: StockOption, ms: MarketState, params: GKParams):
    discount_factor = np.exp(-ms.domestic_rate * (dt(option, ms)))
    foreign_factor = np.exp(-params.foreign_rate * (dt(option, ms)))

    
    call_price = stats.norm.cdf(d1(option, ms, params),0.0,1.0) * foreign_factor * ms.spot_price\
            - stats.norm.cdf(d2(option, ms, params),0.0,1.0) * option.strike_price * discount_factor
    put_price = stats.norm.cdf(-d2(option, ms, params), 0.0,1.0) * option.strike_price * discount_factor\
        - stats.norm.cdf(-d1(option, ms, params), 0.0,1.0) * ms.spot_price * foreign_factor
    
    return np.where(option.is_call, call_price, put_price)

This block is realisation of greeks by formulas under.

In [415]:
def delta(option: StockOption, ms: MarketState, params: GKParams):
    # nd1 = stats.norm.cdf(d1(option, ms, params))*np.exp(-params.foreign_rate * option.expiration_time)
    # return np.where(option.is_call, nd1, nd1 - 1)
    return stats.norm.cdf(d1(option, ms, params))*np.exp(-params.foreign_rate * option.expiration_time)
    


def gamma(option: StockOption, ms: MarketState, params: GKParams):
    return stats.norm.pdf(d1(option, ms, params))*np.exp(-params.foreign_rate * option.expiration_time) / (ms.spot_price * params.volatility * np.sqrt(dt(option, ms)))


def theta(option: StockOption, ms: MarketState, params: GKParams):
    foreign_discont = np.exp(-params.foreign_rate  * (dt(option, ms)))
    a = params.foreign_rate * ms.spot_price * foreign_discont * stats.norm.cdf(d1(option, ms, params))
    b = ms.spot_price * stats.norm.pdf(d1(option, ms, params)) * foreign_discont * params.volatility\
        / (2 * np.sqrt(dt(option, ms)))
    
    d_discount_factor = ms.domestic_rate * np.exp(-ms.domestic_rate * (dt(option, ms)))

    call_theta = a - b - option.strike_price * d_discount_factor * stats.norm.cdf(d2(option, ms, params))
    return call_theta
    # return np.where(option.is_call, call_theta, put_theta)

Now we can launch our model and, at the first, let's set some parametrs. 

In [416]:
spot = 1.0581
strikes = np.array((0.9*spot, 1.1*spot))
matur = 1
dom_rate=2.7
for_rate=3
vol=6

calls = CallStockOption(strike_price=strikes, expiration_time=matur)

ms = MarketState(spot_price=spot, domestic_rate=dom_rate)

params = GKParams(volatility=vol, foreign_rate=for_rate)

print("Price of options: ", price(calls, ms, params))
print("Delta : ", delta(calls, ms, params))
print("Gamma : ", gamma(calls, ms, params))



Price of options:  [0.05252301 0.05250671]
Delta :  [0.04971234 0.0497038 ]
Gamma :  [3.82877988e-05 4.22592126e-05]


Now let describe p.2. We need to solve system of equation:

$$
\begin{cases}
\Delta^{call}_1N^*_1 - \Delta^{call}_2N^*_2 - N^*_3 = 0  \\ 
\Gamma_1N^*_1 - \Gamma_2N^*_2 = 0 \\
N^*_1 + N^*_2 + N^*_3 = N
\end{cases}
$$



In [417]:
N = 1000

first_row = np.append(delta(calls, ms, params), 1)
first_row[1] = -first_row[1]
first_row[2] = -first_row[2]

second_row = np.append(gamma(calls, ms, params), 0)
second_row[1] = -second_row[1]

A = np.array([first_row, 
              second_row,
              [1,1,1]
])
b = np.array([0, 0, N])

propotion_of_portf = np.linalg.solve(A, b) / N
print("Propotions: ", propotion_of_portf)

Propotions:  [0.52336782 0.47418304 0.00244914]


Let's make p.3.

In [418]:
budget = 100000
price_portf = np.append(price(calls, ms, params), spot)
theta_of_options = np.append(theta(calls, ms, params), 0)

amount_in_portfolio = budget / np.sum(price_portf*propotion_of_portf)
propotion_of_portf *= amount_in_portfolio
propotion_of_portf[1] = -propotion_of_portf[1]

theta_portf = np.dot(theta_of_options,propotion_of_portf)

print("Theta of portfolio: ", theta_portf)

Theta of portfolio:  14138.894547928823


At the end, we make fix many of paramaters in the model and this way isn't right , althought volatility is changable. In real life we can get implied vol from model and market data. Also for FX option there are another model(SABR, local vol) for more suffitient result.